In [2]:
%load_ext autoreload
%autoreload 2

from fundData import *
from helper import Helper
import pprint, json, math
import fitz
import pandas as pd
from collections import defaultdict

dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"
# dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
# fund_path = r'C:\Users\rando\OneDrive\Documents\Dec 24'

dry_path = r'\output\DryRun.pdf'
fin_path = r'\files\financial_indices.xlsx'

json_path = dir_path + r'\output\dump.json'
pkl_path = dir_path + r'\output\sample.pkl'

mutual_fund = Helper.get_fund_paths(fund_path)

In [ ]:
#Helper.dump_pickle_data(nested_data, pkl_path)

pkl_data = Helper.load_pickle_data(pkl_path)
object = Tata(dir_path, dry_path, fin_path)

In [ ]:
""" SAMCO PDF FILE MAIN CODE"""

object = Samco(dir_path, dry_path, fin_path)
file_path = mutual_fund['Samco Mutual Fund']


fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']
bbox = object.PARAMS['clip_bbox']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
sample = input("Do you want to proceed (Y/N)?: ")

pages =  [3, 5, 7, 9, 11, 13, 15, 17, 18]

data = object.get_clipped_data(file_path,pages,bbox, fund_titles)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

#CREATE FINAL CONTENT    
# final_text = dict()
# for fund, items in extracted_text.items():
    
#     content_dict = dict()
#     for header, content in items.items():
#         header_content = object.match_regex_to_content(header, content)
#         content_dict.update(header_content)

#     final_text[fund] = content_dict

In [ ]:
""" TATA FILE MAIN CODE """

object = Tata(dir_path, dry_path, fin_path)
file_path = mutual_fund["Tata Mutual Fund"]


fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']
bbox = object.PARAMS['clip_bbox']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)

sample = input("Do you want to proceed (Y/N)?: ")
pages =  [i for i in range(17,84)]

data = object.get_clipped_data(file_path,pages,bbox, fund_titles)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text, unextracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
""" FRANKLIN TEMPLETON FILE MAIN CODE"""

object = FranklinTempleton(dir_path, dry_path, fin_path)
file_path = mutual_fund["Franklin Templeton Mutual Fund"]

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)

pages =  [i for i in range(17,50)]

data = object.extract_data_relative_line(file_path,pages,line_x ,'left', fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
""" BANDHAN MF FILE MAIN CODE"""

object = Bandhan(dir_path, dry_path, fin_path)
file_path = mutual_fund["Bandhan Mutual Fund"]

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
pages =  [i for i in range(17,83)]

data = object.extract_data_relative_line(file_path,pages,line_x ,'left', fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
""" HELIOS MF FILE MAIN CODE"""

object = Helios(dir_path, dry_path, fin_path)
file_path = mutual_fund['Helios Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages =  [2, 4, 6, 8, 10]

data = object.extract_data_relative_line(file_path,pages,line_x ,'left', fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
""" EDELWEISS MP FILE MAIN CODE"""

object = Edelweiss(dir_path, dry_path, fin_path)
file_path = mutual_fund['Edelweiss Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [i for i in range(10,76)]
fund_titles,objectives = object.get_proper_fund_names(file_path,pages)

data = object.extract_data_relative_line(file_path,pages,line_x ,'right', fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [8]:
""""HSBC MAIN FILE CODE""" #Issue: Some pages have 2 or more fund data on same page
#some data is present on right side as well so define two lines

object = HSBC(dir_path,dry_path,fin_path)
file_path = mutual_fund['HSBC Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
data = object.extract_data_relative_line(file_path,pages,line_x ,'right', fund_titles) #used line here
data = object.extract_span_data(data,[])

clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
""" INVESCO MF MAIN FILE CODE""" #Issue: two fund on same page

object = Invesco(dir_path,dry_path,fin_path)
file_path = mutual_fund['Invesco Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [i for i in range(4,44)]
data = object.get_clipped_data(file_path,pages,bbox, fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [27]:
""" ITI MAIN FILE CODE""" #Issues: NO so far

object = ITI(dir_path,dry_path,fin_path)
file_path = mutual_fund['ITI Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [i for i in range(9,27)]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""JM FUND MAIN FILE CODE""" #objective on left, sometimes date is with header

object = JMMF(dir_path,dry_path,fin_path)
file_path = mutual_fund['JM Financial Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")
pages = [22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
"""DSP MAIN FILE CODE""" #Issue: highlight count must be lower
#Issue: data on both side ends so issue coming

object = DSP(dir_path,dry_path,fin_path)
file_path = mutual_fund['DSP Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [ i for i in range(1,66)]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
"""KOTAK FUND MAIN CODE""" #Issue: 1 or more fund on same page, color on headers
object = Kotak(dir_path,dry_path,fin_path)
file_path = mutual_fund['Kotak Mahindra Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 36, 37, 38, 40, 41, 43, 44, 46, 48, 49, 50, 52, 54, 55, 56, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107]
data = object.extract_data_relative_line(file_path,pages,line_x,'left',fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
#ISSUE: THE FUND NAMES ARE IMAGES """ LIC MF MAIN CODE"""
 #Issue: objective on right of line

object = LIC(dir_path,dry_path,fin_path)
file_path = mutual_fund['Kotak Mahindra Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)


In [24]:
""" MAHINDRA MANULIFE MAIN CODE""" #incrase pages count

object = MahindraManu(dir_path,dry_path,fin_path)
file_path = mutual_fund['Mahindra Manulife Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")


fund_titles = object.get_proper_fund_names(file_path,[i for i in range(1,80)])
pages = [i for i in range(8,34)]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
""" MOTILAL OSWAL MAIN CODE FILE"""

object = MotilalOswal(dir_path,dry_path,fin_path)
file_path = mutual_fund['Motilal Oswal Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [i for i in range(4,21)]
data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""NJMF MAIN CODE FILE"""

object = NJMF(dir_path,dry_path,fin_path)
file_path = mutual_fund['NJ Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [1, 5, 7, 9]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""QUANT MAIN CODE FILE"""

object = QuantMF(dir_path,dry_path,fin_path)
file_path = mutual_fund['Quant Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 61, 63, 65]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
pages = [12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 61, 63, 65]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""SUNDARAM MAIN FILE CODE"""

object = Sundaram(dir_path,dry_path,fin_path)
file_path = mutual_fund['Sundaram Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""TAURUS MAIN FILE CODE"""

object = Taurus(dir_path,dry_path,fin_path)
file_path = mutual_fund['Taurus Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [13, 14, 15, 16, 17, 19, 20]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""TRUST MAIN FILE CODE""" #Issue: Invest Obj in right #Names coming as headers fund managers

object = Trust(dir_path,dry_path,fin_path)
file_path = mutual_fund['Trust Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [6, 7, 8, 9, 10, 11, 12, 13]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""UTI MAIN FILE CODE"""

object = UTI(dir_path,dry_path,fin_path)
file_path = mutual_fund['UTI Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 41, 43, 45, 46, 48, 50, 52, 54, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""WHITEOAK MAIN FILE CODE""" #Issue: Inv Obj in the right
object = WhiteOak(dir_path,dry_path,fin_path)
file_path = mutual_fund['WhiteOak Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [7, 9, 11, 13, 15, 17, 19, 20, 21, 23, 25, 26, 28, 30, 32, 34, 35]
data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
'''' 360 ONE MAIN FILE CODE'''

object = ThreeSixtyOne(dir_path,dry_path,fin_path)
file_path = mutual_fund['360 ONE Mutual Fund']
fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_bbox']
data_cond = object.PARAMS['data']

# print(fund_data,line_x,bbox,data_cond)
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [5, 6, 7, 8, 9, 10, 11, 12]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
""" BAJAJ FINSERV MAIN FILE CODE"""

object = BajajFinServ(dir_path,dry_path,fin_path)
file_path = mutual_fund['Bajaj finserv Mutual Fund']
fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_bbox']
data_cond = object.PARAMS['data']

# print(fund_data,line_x,bbox,data_cond)
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [15, 17, 19, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33]

data_l = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used line here
data_r = object.extract_data_relative_line(file_path,pages,line_x,"right",fund_titles) #used line here

data = object.combine_left_right_data([data_l,data_r])
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond)
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
""" BARODA BNP MAIN FILE CODE""" #Issue: Header funds

object = BarodaBNP(dir_path,dry_path,fin_path)
file_path = mutual_fund['Baroda BNP Paribas Mutual Fund']
fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_bbox']
data_cond = object.PARAMS['data']

# print(fund_data,line_x,bbox,data_cond)
# path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
# msgs = input("\nEnter Y/N: ")

pages = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]

fund_titles = object.get_proper_fund_names(file_path, pages,(0,0,210,75))

data = object.extract_data_relative_line(file_path,pages,line_x,'left', fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
"""NAVI MAIN FILE CODE""" #Issue: Left and right

object = NAVI(dir_path,dry_path,fin_path)
file_path = mutual_fund['Navi Mutual Fund']
fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_bbox']
data_cond = object.PARAMS['data']

# print(fund_data,line_x,bbox,data_cond)
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [2, 6, 8, 10, 13, 16, 18, 20, 22]
data_l = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used line here
data_r = object.extract_data_relative_line(file_path,pages,line_x,"right",fund_titles) #used line here

data = object.combine_left_right_data([data_l,data_r])
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond)
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""ZERODHA MAIN CODE""" #Issue left and right

object = Zerodha(dir_path,dry_path,fin_path)
file_path = mutual_fund['Zerodha Mutual Fund']
fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_bbox']
data_cond = object.PARAMS['data']

# print(fund_data,line_x,bbox,data_cond)
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [3, 5, 7, 9, 11, 13, 15]
data_l = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used line here
data_r = object.extract_data_relative_line(file_path,pages,line_x,"right",fund_titles) #used line here

data = object.combine_left_right_data([data_l,data_r])
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond)
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 14.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
for fund, content in extracted_text.items():
    
    print(f"\n---{fund}---")
    for i in content.keys():
        print(i)

In [1]:
Helper.quick_json_dump(extracted_text,json_path)       

NameError: name 'extracted_text' is not defined